##### Copyright 2018 The TensorFlow Authors.

In [0]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [0]:
#@title MIT License
#
# Copyright (c) 2017 François Chollet
#
# Permission is hereby granted, free of charge, to any person obtaining a
# copy of this software and associated documentation files (the "Software"),
# to deal in the Software without restriction, including without limitation
# the rights to use, copy, modify, merge, publish, distribute, sublicense,
# and/or sell copies of the Software, and to permit persons to whom the
# Software is furnished to do so, subject to the following conditions:
#
# The above copyright notice and this permission notice shall be included in
# all copies or substantial portions of the Software.
#
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL
# THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING
# FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER
# DEALINGS IN THE SOFTWARE.

# Entrena tu primer red neuronal: clasificación básica

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/keras/basic_classification"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />Ver en TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/keras/basic_classification.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Correr en Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/keras/basic_classification.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />Ver código fuente en GitHub</a>
  </td>
</table>

Esta guía entrena un modelo de red neuronal para clasificar imágenes de prendas de vestir, como zapatillas y remeras. Está bien si no comprendes todos los detalles al principio; este es un vistazo general de un programa completo en Tensorflow con los detalles explicados en el camino.

Esta guía usa [tf.keras](https://www.tensorflow.org/guide/keras), una API de alto nivel para construir y entrenar modelos en Tensorflow.

In [0]:
from __future__ import absolute_import, division, print_function

# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

## Importa el Fashion MNIST dataset

Esta guía usa el [Fashion MNIST](https://github.com/zalandoresearch/fashion-mnist) dataset, que contiene 70.000 imágenes en blanco y negro de 10 categorías. Estas imágenes muestran prendas de vestir individuales en baja resolución (28 por 28 píxeles), como las siguientes:

<table>
  <tr><td>
    <img src="https://tensorflow.org/images/fashion-mnist-sprite.png"
         alt="Fashion MNIST sprite"  width="600">
  </td></tr>
  <tr><td align="center">
    <b>Figure 1.</b> <a href="https://github.com/zalandoresearch/fashion-mnist">Fashion-MNIST samples</a> (by Zalando, MIT License).<br/>&nbsp;
  </td></tr>
</table>

El dataset Fashion MNIST está diseñado para ser un reemplazo automático del clásico [MNIST](http://yann.lecun.com/exdb/mnist/) dataset—muchas veces utilizado como el "Hola, Mundo" del machine learning para computer vision. Éste contiene imágenes de dígitos (0, 1, 2, etc) escritos a mano en un formato idéntico al de las prendas de vestir que vamos a utilizar en este tutorial.

Esta guía usa Fashion MNIST para mayor variedad, porque es un problema un poco más desafiante que el MNIST común. Ambos datasets son relativamente pequeños y son utilizados para verificar que un algoritmo funciona como se espera. Son buenos puntos de partida para probar código.

Utilizaremos 60.000 imágenes para entrenar la red y 10.000 imágenes para evaluar cúan exactamente aprendió la misma a clasificar imágenes. Puedes acceer al Fashion MNIST directamente de Tensorflow, sólo importa y carga los datos:

In [0]:
fashion_mnist = keras.datasets.fashion_mnist

(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

Cargar el dataset devuelve cuatro arrays de NumPy:

* Los arrays `train_images` y `train_labels` son el *set de entrenamiento*—los datos que el modelo usa para aprender.
* El modelo se prueba contra el *set de testing*, los arrays `test_images` y `test_labels`.

Las imágenes son arrays de NumPy de dimensiones 28x28, con valores de pixels entre 0 y 255. Las etiquetas o *labels* son un array de enteros, con valores entre 0 y 9. Estos corresponden a la *clase* de prenda de vestir que la imágen representa.

<table>
  <tr>
    <th>Label</th>
    <th>Clase</th> 
  </tr>
  <tr>
    <td>0</td>
    <td>Remera/top</td> 
  </tr>
  <tr>
    <td>1</td>
    <td>Pantalones</td> 
  </tr>
    <tr>
    <td>2</td>
    <td>Pullover</td> 
  </tr>
    <tr>
    <td>3</td>
    <td>Vestido</td> 
  </tr>
    <tr>
    <td>4</td>
    <td>Saco/campera</td> 
  </tr>
    <tr>
    <td>5</td>
    <td>Sandalia</td> 
  </tr>
    <tr>
    <td>6</td>
    <td>Camisa</td> 
  </tr>
    <tr>
    <td>7</td>
    <td>Zapatilla</td> 
  </tr>
    <tr>
    <td>8</td>
    <td>Bolso/mochila</td> 
  </tr>
    <tr>
    <td>9</td>
    <td>Bota</td> 
  </tr>
</table>

Cada imagen tiene asignada una sola label. Como los nombres de las clases no están incluídos con el dataset, guárdalos aquí para usarlos al momento de dibujar las imágenes:

In [0]:
class_names = ['Remera/top', 'Pantalones', 'Pullover', 'Vestido', 'Saco/campera', 
               'Sandalia', 'Camisa', 'Zapatilla', 'Bolso/mochila', 'Bota']

## Explora los datos

Vamos a explorar el formato del dataset antes de comenzar a entrenar el modelo. A continuación se muestra que hay 60.000 imágenes en el set de entrenamiento, con cada imagen representada con 28x28 pixels:

In [0]:
train_images.shape

De la misma manera, hay 60.000 labels en el set de entrenamiento:

In [0]:
len(train_labels)

Cada label es un entero entre 0 y 9:

In [0]:
train_labels

Hay 10.000 imágenes en el set de testing. De nuevo, cada imagen está representada por 28x28 pixels:

In [0]:
test_images.shape

Y el set de testing contiene 10.000 labels de imágenes:

In [0]:
len(test_labels)

## Preprocesa los datos

Los datos deben ser preprocesados antes de entrenar la red. Si miras la primer imagen del set de entrenamiento, podrás ver que los alores de los pixels están en el rango de 0 a 255:

In [0]:
plt.figure()
plt.imshow(train_images[0])
plt.colorbar()
plt.grid(False)
plt.show()

Pasamos estos valores a un rango entre 0 a 1 antes de alimentarlos a la red neuronal. Para esto, dividimos los valores por 255. Es importante que el *set de entrenamiento* y el *set de testing* sean preprocesados de la misma manera:

In [0]:
train_images = train_images / 255.0

test_images = test_images / 255.0

Muestra las primeras 25 imágenes del *set de entrenamiento* y el nombre de clase debajo de cada imagen. Verifica que la data está en el formato correcto y estamos listos para construir y entrenar la red.

In [0]:
plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(train_images[i], cmap=plt.cm.binary)
    plt.xlabel(class_names[train_labels[i]])
plt.show()

## Construye el modelo

Construir la red neuronal requiere configurar las capas del modelo, y luego compilarlo.

### Configura las capas

El cimiento básico de una red neuronal son las capas o *layers*. Una layer extrae representaciones de la data con la que fue alimetada y, con suerte, estas representaciones sean útiles y tengan sentido para el problema que intentamos resolver.

El deep learning consiste mayormente en encadenar layers simples entre sí. La mayoría de las capas, como `tf.keras.layers.Dense`, tienen parámetros que se aprenden durante el entrenamiento.

In [0]:
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(28, 28)),
    keras.layers.Dense(128, activation=tf.nn.relu),
    keras.layers.Dense(10, activation=tf.nn.softmax)
])

La primer layer en esta red, `tf.keras.layers.Flatten`, transforma el formato de las imágenes de un array 2d (de 28 por 28 pixels) a un array de 1 dimensión de 28 * 28 = 784 pixels. Puedes pensar en esta layer como "desapilar" filas de pixels de la imagen y alineándolas. Esta layer no tiene parámetros para aprender; solamente cambia el formato de los datos.

Luego de que los pixels son aplanados, la red consiste de una secuencia de dos `tf.keras.layers.Dense` layers. Estas son capas neuronales densa o completamente conectadas. La primera `Dense` layer tiene 128 nodos (o neuronas). La segunda (y última) layer es una layer *softmax* de 10 nodos. Esta devuelve un array de 10 probabilidades cuya suma es 1. Cada nodo contiene un puntaje que indica la probabilidad de que la imagen analizada corresponda a cada una de las 10 clases.

### Compila el modelo

Antes de que el modelo esté listo para el entrenamiento, necesita algunas configuraciones más. Estas son agregadas durante el paso de *compilación*:

* *Loss function* —Función que mide cuán exacto es el modelo durante el entrenamiento. Queremos minimizar el resultado de esta función para *mover* el modelo en la dirección correcta.
* *Optimizer* —La manera en la que el modelo es actualizado en base a los datos que ve y la loss function.
* *Metrics* —Métricas a utilizar para monitorear el entrenamiento y pasos de testing. El siguiente ejemplo usa la exactitud o "accuracy", la parte de las imágenes que son clasificadas correctamente.

In [0]:
model.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

## Entrena el modelo

Entrenar el modelo de red neuronal requiere los siguientes pasos:

1. Alimenta los datos de entrenamiento al modelo. EN este ejemplo, los arrays `train_images` y `train_labels`.
2. El modelo aprende a asociar imágenes a labels.
3. Le pedimos al modelo que haga sus predicciones en base a un set de testing. En este ejemplo, el array `test_images`. Verificamos que las predicciones concuerden con las labels del array `test_labels`.

Para comenzar el entrenamiento, llama al método `model.fit`—el modelo es "ajustado" a los datos de entrenamiento:

In [0]:
model.fit(train_images, train_labels, epochs=5)

As the model trains, the loss and accuracy metrics are displayed. This model reaches an accuracy of about 0.88 (or 88%) on the training data.

## Evaluate accuracy

Next, compare how the model performs on the test dataset:

In [0]:
test_loss, test_acc = model.evaluate(test_images, test_labels)

print('Test accuracy:', test_acc)

It turns out, the accuracy on the test dataset is a little less than the accuracy on the training dataset. This gap between training accuracy and test accuracy is an example of *overfitting*. Overfitting is when a machine learning model performs worse on new data than on their training data. 

## Make predictions

With the model trained, we can use it to make predictions about some images.

In [0]:
predictions = model.predict(test_images)

Here, the model has predicted the label for each image in the testing set. Let's take a look at the first prediction:

In [0]:
predictions[0]

A prediction is an array of 10 numbers. These describe the "confidence" of the model that the image corresponds to each of the 10 different articles of clothing. We can see which label has the highest confidence value:

In [0]:
np.argmax(predictions[0])

So the model is most confident that this image is an ankle boot, or `class_names[9]`. And we can check the test label to see this is correct:

In [0]:
test_labels[0]

We can graph this to look at the full set of 10 channels

In [0]:
def plot_image(i, predictions_array, true_label, img):
  predictions_array, true_label, img = predictions_array[i], true_label[i], img[i]
  plt.grid(False)
  plt.xticks([])
  plt.yticks([])
  
  plt.imshow(img, cmap=plt.cm.binary)

  predicted_label = np.argmax(predictions_array)
  if predicted_label == true_label:
    color = 'blue'
  else:
    color = 'red'
  
  plt.xlabel("{} {:2.0f}% ({})".format(class_names[predicted_label],
                                100*np.max(predictions_array),
                                class_names[true_label]),
                                color=color)

def plot_value_array(i, predictions_array, true_label):
  predictions_array, true_label = predictions_array[i], true_label[i]
  plt.grid(False)
  plt.xticks([])
  plt.yticks([])
  thisplot = plt.bar(range(10), predictions_array, color="#777777")
  plt.ylim([0, 1]) 
  predicted_label = np.argmax(predictions_array)
 
  thisplot[predicted_label].set_color('red')
  thisplot[true_label].set_color('blue')

Let's look at the 0th image, predictions, and prediction array. 

In [0]:
i = 0
plt.figure(figsize=(6,3))
plt.subplot(1,2,1)
plot_image(i, predictions, test_labels, test_images)
plt.subplot(1,2,2)
plot_value_array(i, predictions,  test_labels)
plt.show()

In [0]:
i = 12
plt.figure(figsize=(6,3))
plt.subplot(1,2,1)
plot_image(i, predictions, test_labels, test_images)
plt.subplot(1,2,2)
plot_value_array(i, predictions,  test_labels)
plt.show()

Let's plot several images with their predictions. Correct prediction labels are blue and incorrect prediction labels are red. The number gives the percent (out of 100) for the predicted label. Note that it can be wrong even when very confident. 

In [0]:
# Plot the first X test images, their predicted label, and the true label
# Color correct predictions in blue, incorrect predictions in red
num_rows = 5
num_cols = 3
num_images = num_rows*num_cols
plt.figure(figsize=(2*2*num_cols, 2*num_rows))
for i in range(num_images):
  plt.subplot(num_rows, 2*num_cols, 2*i+1)
  plot_image(i, predictions, test_labels, test_images)
  plt.subplot(num_rows, 2*num_cols, 2*i+2)
  plot_value_array(i, predictions, test_labels)
plt.show()

Finally, use the trained model to make a prediction about a single image. 

In [0]:
# Grab an image from the test dataset
img = test_images[0]

print(img.shape)

`tf.keras` models are optimized to make predictions on a *batch*, or collection, of examples at once. So even though we're using a single image, we need to add it to a list:

In [0]:
# Add the image to a batch where it's the only member.
img = (np.expand_dims(img,0))

print(img.shape)

Now predict the image:

In [0]:
predictions_single = model.predict(img)

print(predictions_single)

In [0]:
plot_value_array(0, predictions_single, test_labels)
plt.xticks(range(10), class_names, rotation=45)
plt.show()

`model.predict` returns a list of lists, one for each image in the batch of data. Grab the predictions for our (only) image in the batch:

In [0]:
prediction_result = np.argmax(predictions_single[0])
print(prediction_result)

And, as before, the model predicts a label of 9.